# Molecule from smiles: 

In [ ]:
import mbuild as mb

In [2]:
from functions import MonToSmiles
from functions.MonToSmiles import mon_to_smiles
from mon_classes import P3HT

smiles = mon_to_smiles(fragment=P3HT())[1]
mon_dim = mon_to_smiles(fragment=P3HT())[0]

mon_dim.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
from functions import EspalomaFxn
from functions.EspalomaFxn import espaloma

typed_mol2 = '/Users/madilyn/Projects/repos/forcefields/typed_mol2/p3ht_test.mol2'
xml_filepath = '/Users/madilyn/Projects/repos/forcefields/xml_files/p3ht_test.xml'

espaloma(SMILES=smiles,
        XML_FILEPATH=xml_filepath,
        TYPED_FILEPATH=typed_mol2)

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


Done!


1 molecule converted


<Compound 27 particles, 27 bonds, non-periodic, id: 5843365744>